In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn import tree
from collections import Counter

In [2]:
data_set_complete = pd.read_csv('data_set_complete.csv', parse_dates=['fechapostulacion', 'ultima_vista'], dtype={'sepostulo':'int8', 'idaviso':'int64', 'idpostulante':'object', 'veces_visto':'int16', 'estudios':'category', 'estado':'category', 'edad':'int8', 'sexo':'category','nombre_zona':'category', 'tipo_de_trabajo':'category', 'nivel_laboral':'category', 'nombre_area':'category'})
data_set_complete.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13208286 entries, 0 to 13208285
Data columns (total 16 columns):
sepostulo               int8
idaviso                 int64
idpostulante            object
fechapostulacion        datetime64[ns]
ultima_vista            datetime64[ns]
veces_visto             int16
estudios                category
estado                  category
sexo                    category
edad                    int8
titulo                  object
nombre_zona             category
tipo_de_trabajo         category
nivel_laboral           category
nombre_area             category
denominacion_empresa    object
dtypes: category(7), datetime64[ns](2), int16(1), int64(1), int8(2), object(3)
memory usage: 755.8+ MB


In [ ]:
data = data_set_complete.sample(1000000)
del data_set_complete

In [ ]:
data['nombre_zona'].fillna(value='Gran Buenos Aires',inplace=True)
data['tipo_de_trabajo'].fillna(value='Full-time',inplace=True)
data['nivel_laboral'].fillna(value='Senior / Semi-Senior',inplace=True)
data['nombre_area'].fillna(value='Ventas',inplace=True)
data['estado'].fillna(value='Graduado',inplace=True)
data['estudios'].fillna(value='Secundario',inplace=True)

In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000000 entries, 9656002 to 476313
Data columns (total 16 columns):
sepostulo               1000000 non-null int8
idaviso                 1000000 non-null int64
idpostulante            1000000 non-null object
fechapostulacion        499865 non-null datetime64[ns]
ultima_vista            134827 non-null datetime64[ns]
veces_visto             1000000 non-null int16
estudios                1000000 non-null category
estado                  1000000 non-null category
sexo                    1000000 non-null category
edad                    1000000 non-null int8
titulo                  937435 non-null object
nombre_zona             1000000 non-null category
tipo_de_trabajo         1000000 non-null category
nivel_laboral           1000000 non-null category
nombre_area             1000000 non-null category
denominacion_empresa    937385 non-null object
dtypes: category(7), datetime64[ns](2), int16(1), int64(1), int8(2), object(3)
memory usage: 6

In [35]:
def df_encode_categorical_vars(df):
    for column in df.columns:
        if (df[column].dtype.name == 'category'):
            le = preprocessing.LabelEncoder()
            df[column] = le.fit_transform(df[column].astype(str))

In [36]:
df_encode_categorical_vars(data)
data.head()

,sepostulo,idaviso,idpostulante,fechapostulacion,ultima_vista,veces_visto,estudios,estado,sexo,edad,titulo,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
9656002,0,1112033906,Np8Q4,NaT,NaT,0,6,2,1,42,Cajero/a-Repositor/a (Capital Federal),5,1,4,179,Farmacity
5126576,1,1112237543,ekoYaeE,2018-01-23 09:59:03,NaT,1,5,2,1,23,Ejecutivos de Cuenta JR canal Farmacia,5,1,2,29,DNM FARMA S.A
274586,1,1111753928,1QPxbXB,2018-02-05 09:38:51,NaT,1,4,2,0,21,CAJEROS (Caba) - Eventual,5,1,2,170,BAYTON
11648245,0,1112355178,eb58j2,NaT,NaT,0,5,2,1,36,Maquilladoras Profesionales,5,1,4,179,Adecco -Sales & Marketing
3112641,1,1112267958,NzjQva5,2018-01-18 07:37:18,NaT,1,4,2,1,24,Operario de producción con experiencia en Mont...,5,1,4,139,Pullmen Servicios Empresarios S.A.


In [37]:
train_set, test_set, train_classes, test_classes = train_test_split(data.drop(columns=['sepostulo','idaviso','idpostulante', 'fechapostulacion','ultima_vista','titulo','denominacion_empresa']), data['sepostulo'], test_size=0.3)

In [43]:
clf = tree.DecisionTreeClassifier().fit(train_set, train_classes)
clf.score(test_set, test_classes)

0.9989466666666666

In [ ]:
test100k=pd.read_csv('test_final_100k_complete.csv', parse_dates=['fechapostulacion', 'ultima_vista'], dtype={'sepostulo':'int8', 'idaviso':'int64', 'idpostulante':'object', 'veces_visto':'int16', 'estudios':'category', 'estado':'category', 'edad':'int8', 'sexo':'category','nombre_zona':'category', 'tipo_de_trabajo':'category', 'nivel_laboral':'category', 'nombre_area':'category'})
test100k['nombre_zona'].fillna(value='Gran Buenos Aires',inplace=True)
test100k['tipo_de_trabajo'].fillna(value='Full-time',inplace=True)
test100k['nivel_laboral'].fillna(value='Senior / Semi-Senior',inplace=True)
test100k['nombre_area'].fillna(value='Ventas',inplace=True)
test100k['estado'].fillna(value='Graduado',inplace=True)
test100k['estudios'].fillna(value='Secundario',inplace=True)
df_encode_categorical_vars(test100k)
test100k.info()

In [82]:
test100k['sepostulo'] = clf.predict(test100k.drop(columns=['id','sepostulo','idaviso','idpostulante', 'fechapostulacion','ultima_vista','titulo','denominacion_empresa']))
test100k[['id','sepostulo']].head()

,id,sepostulo
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [83]:
test100k['sepostulo'].mean()

0.34086

In [84]:
test100k[['id','sepostulo']].to_csv('Resultados/tree_confillna_convecesvisto.csv', index=False)

In [59]:
data['sexo'].value_counts()

0    511843
1    470657
2     17500
Name: sexo, dtype: int64

In [61]:
data['sexo'] = data['sexo'].apply(lambda x: 0 if (x==2) else x)

In [64]:
train_set, test_set, train_classes, test_classes = train_test_split(data.drop(columns=['sepostulo','idaviso','idpostulante', 'fechapostulacion','ultima_vista','titulo','denominacion_empresa']), data['sepostulo'], test_size=0.4)

In [65]:
clf2 = tree.DecisionTreeClassifier().fit(train_set, train_classes)
clf2.score(test_set, test_classes)

0.9991825

In [66]:
train_set, test_set, train_classes, test_classes = train_test_split(data.drop(columns=['veces_visto','sepostulo','idaviso','idpostulante', 'fechapostulacion','ultima_vista','titulo','denominacion_empresa']), data['sepostulo'], test_size=0.4)

In [67]:
clf3 = tree.DecisionTreeClassifier().fit(train_set, train_classes)
clf3.score(test_set, test_classes)

0.6125825

- CLARAMENTE VEMOS QUE EL OVERFITTING ME LO CAUSABA EL CAMPO VECES_VISTO, PORQUE COMO LAS NO POSTULACIONES, CASI TODAS TENIAN VECES_VISTO EN 0, Y LAS POSTULACIONES EN 1 O MAS, PRACTICAMENTE QUE EL FEATURE FUNDAMENTAL DE LA DIVISION DE CLASES LO TOMABA COMO VECES_VISTO

In [75]:
test100k.head()

,id,idaviso,idpostulante,fechapostulacion,ultima_vista,veces_visto,estudios,estado,sexo,edad,titulo,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,sepostulo
0,0,739260,6M9ZQR,NaT,NaT,0,0,2,0,42,Asistente Comercial,1,1,1,29,BUMERAN.COM ARGENTINA,0
1,1,739260,6v1xdL,NaT,NaT,0,5,2,1,30,Asistente Comercial,1,1,1,29,BUMERAN.COM ARGENTINA,0
2,2,739260,ezRKm9,NaT,NaT,0,6,2,0,36,Asistente Comercial,1,1,1,29,BUMERAN.COM ARGENTINA,0
3,3,758580,1Q35ej,NaT,NaT,0,3,2,1,68,Oracle Financials Sr. Application Developer,1,1,3,164,TeleTech,0
4,4,758580,EAN4J6,NaT,NaT,0,6,1,0,32,Oracle Financials Sr. Application Developer,1,1,3,164,TeleTech,0


In [68]:
test100k['sexo'].value_counts()

1    49226
0    48039
2     2735
Name: sexo, dtype: int64

In [78]:
test100k['sexo'] = test100k['sexo'].apply(lambda x: 1 if (x==2) else x)
test100k['sepostulo'] = clf3.predict(test100k.drop(columns=['id','sepostulo','veces_visto','idaviso','idpostulante','fechapostulacion','ultima_vista','titulo','denominacion_empresa']))
test100k['sepostulo'].mean()

0.44906

In [80]:
test100k[['id','sepostulo']].to_csv('Resultados/tree_confillna_sinvecesvisto.csv', index=False)

In [34]:
Counter(" ".join(data["titulo"].astype(str)).split()).most_common(100)[1]

[('de', 469730),
 ('-', 236679),
 ('para', 113055),
 ('Analista', 76133),
 ('y', 67859),
 ('nan', 62565),
 ('DE', 60855),
 ('en', 59389),
 ('al', 50044),
 ('Administrativo', 49606),
 ('Zona', 44277),
 ('/', 39307),
 ('Recepcionista', 37451),
 ('Atención', 36004),
 ('Operario', 35983),
 ('Asistente', 33395),
 ('Vendedor', 32448),
 ('Part', 30268),
 ('Time', 29568),
 ('con', 28773),
 ('CABA', 26663),
 ('Representante', 23051),
 ('Administrativa', 22985),
 ('Ventas', 22694),
 ('Call', 21949),
 ('Operarios', 21457),
 ('Comercial', 19540),
 ('Cliente', 19257),
 ('cliente', 19008),
 ('Contable', 18568),
 ('Center', 18428),
 ('Importante', 17445),
 ('Capital', 17356),
 ('Operador', 17185),
 ('a', 16805),
 ('Técnico', 16346),
 ('PARA', 16145),
 ('Federal', 15949),
 ('Personal', 15311),
 ('Y', 13667),
 ('Ejecutivo', 13086),
 ('Vendedores', 13082),
 ('Producción', 12425),
 ('experiencia', 12212),
 ('Jefe', 11822),
 ('Supervisor', 11639),
 ('importante', 11628),
 ('Asesores', 11348),
 ('GBA', 113